# Weekly Stock Return Calculator with Predictions

📈 本工具可計算多檔美股最近一個完整週（週一開盤~週五收盤）的報酬率，並根據使用者預測（看漲或看跌）調整報酬率。
📦 支援直接輸入多個股票代碼並指定預測，線上即時回傳統計結果。
請先安裝以下相關套件：
- `yfinance`
- `pandas`

執行以下命令安裝（若尚未安裝）：
```bash
!pip install yfinance pandas --quiet
```

In [1]:
# 安裝套件（若需要）
!pip install yfinance pandas --quiet

執行以下程式之後，下方會出現如下圖示。  
輸入標的後，在下拉選單選擇看漲(bullish)或看跌(bearish)選項，依序設定完即可按下綠色執行按鈕計算。
![示範圖](img/example.png)

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from IPython.display import display
import ipywidgets as widgets

def get_last_full_week():
    """取得最近一個完整的美股交易週（週一到週五）"""
    today = datetime.now(ZoneInfo("Asia/Shanghai"))  # 明確指定中國時區
    weekday = today.weekday()
    # 若今天為週末，自動回推到週五
    if weekday >= 5:
        today = today - timedelta(days=weekday - 4)
    last_monday = today - timedelta(days=today.weekday())
    last_friday = last_monday + timedelta(days=4)
    return last_monday.date(), last_friday.date()

def calc_weekly_return(symbols, predictions):
    start_date, end_date = get_last_full_week()
    results = []
    for symbol in symbols:
        try:
            df = yf.download(symbol, start=start_date, end=end_date + timedelta(days=1), interval='1d', progress=False, auto_adjust=False)
            df = df.loc[df.index.dayofweek < 5]  # 過濾週末
            if len(df) < 2 or df.index[0].date() > start_date or df.index[-1].date() < end_date:
                results.append({
                    'symbol': symbol,
                    'open': None,
                    'close': None,
                    'change_pct': None,
                    'msg': '資料不足或非美股',
                    'prediction': predictions.get(symbol, 'none'),
                    'prediction_result': 'N/A'
                })
                continue
            monday_open = df.iloc[0]['Open'].iloc[0]  # 從 Series 取第一個值
            friday_close = df.iloc[-1]['Close'].iloc[0]  # 從 Series 取第一個值
            change_pct = (friday_close - monday_open) / monday_open * 100
            pred = predictions.get(symbol, 'none')
            pred_result = 'N/A'
            if pred == 'bullish' and change_pct > 0:
                pred_result = 'Correct'
            elif pred == 'bearish' and change_pct < 0:
                pred_result = 'Correct'
            elif pred in ['bullish', 'bearish']:
                pred_result = 'Wrong'
            results.append({
                'symbol': symbol,
                'open': monday_open,
                'close': friday_close,
                'change_pct': change_pct,
                'msg': '',
                'prediction': pred,
                'prediction_result': pred_result
            })
        except Exception as e:
            results.append({
                'symbol': symbol,
                'open': None,
                'close': None,
                'change_pct': None,
                'msg': f'下載資料失敗: {str(e)}',
                'prediction': predictions.get(symbol, 'none'),
                'prediction_result': 'N/A'
            })
            continue
    return results, start_date, end_date

def show_result(symbols, predictions):
    results, start_date, end_date = calc_weekly_return(symbols, predictions)
    if not results:
        print("無有效數據可顯示")
        return

    df = pd.DataFrame(results)
    df_show = df[['symbol', 'open', 'close', 'change_pct', 'msg', 'prediction', 'prediction_result']]
    df_show.columns = ['股票代碼', '週一開盤價', '週五收盤價', '漲跌幅(%)', '備註', '預測', '預測結果']
    df_show['週一開盤價'] = df_show['週一開盤價'].round(2)
    df_show['週五收盤價'] = df_show['週五收盤價'].round(2)
    df_show['漲跌幅(%)'] = df_show['漲跌幅(%)'].round(3)

    display(df_show)

    valid_returns = df['change_pct'].dropna()
    print(f"\n查詢區間：{start_date.strftime('%Y年%m月%d日')} ~ {end_date.strftime('%Y年%m月%d日')}")
    if valid_returns.empty:
        print("本次查詢沒有有效報酬率可加總")
    else:
        adjusted_returns = []
        for index, row in df.iterrows():
            if pd.notna(row['change_pct']) and row['prediction'] in ['bullish', 'bearish']:
                if row['prediction'] == 'bearish':
                    adjusted_returns.append(row['change_pct'] * -1)
                else:  # bullish
                    adjusted_returns.append(row['change_pct'])
        adjusted_total_return = sum(adjusted_returns) if adjusted_returns else 0
        correct_pred_returns = df[df['prediction_result'] == 'Correct']['change_pct'].dropna().sum()

        print(f"調整後總報酬率（根據預測）：{adjusted_total_return:.2f}%")
        if pd.notna(correct_pred_returns) and correct_pred_returns != 0:
            print(f"預測正確的報酬率總和：{correct_pred_returns:.2f}%")

    if df['change_pct'].isnull().any():
        print("\n有標的資料缺漏（如非美股、停牌、代碼錯誤），詳見『備註』欄.")

# --- 互動式 Widget ---

# 股票代碼輸入
symbols_input = widgets.Text(
    value='',
    placeholder='請以逗號分隔輸入股票代碼（如 AAPL,AVAV,TSLA,JPM,COIN）',
    description='股票代碼:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# 預測輸入（下拉選單）
prediction_widgets = {}
output = widgets.Output()

def update_predictions(*args):
    global prediction_widgets
    syms = [s.strip().upper() for s in symbols_input.value.split(',') if s.strip()]
    
    # 移除不再存在的股票預測
    for sym in list(prediction_widgets.keys()):
        if sym not in syms:
            prediction_widgets.pop(sym)
    
    # 添加或更新新的股票預測
    for sym in syms:
        if sym not in prediction_widgets:
            prediction_widgets[sym] = widgets.Dropdown(
                options=['none', 'bullish', 'bearish'],
                value='none',
                description=f'{sym} 預測:',
                style={'description_width': 'initial'}
            )
    
    # 更新顯示
    with output:
        output.clear_output()
        if syms:
            display(widgets.VBox(list(prediction_widgets.values())))
        else:
            print("請輸入至少一個股票代碼以設定預測。")

# 移除舊的 observe 事件（避免累積）
try:
    symbols_input.unobserve(update_predictions, 'value')
except:
    pass
symbols_input.observe(update_predictions, 'value')

# 計算按鈕
run_button = widgets.Button(description='計算本週投報率', button_style='success')

def on_button_click(b):
    with output:
        output.clear_output()
        syms = [s.strip().upper() for s in symbols_input.value.split(',') if s.strip()]
        if not syms:
            print("請輸入至少一個股票代碼。")
            return
        # 收集預測
        predictions = {sym: pred.value for sym, pred in prediction_widgets.items() if sym in syms}
        show_result(syms, predictions)

run_button.on_click(on_button_click)

# 顯示介面
display(widgets.VBox([symbols_input, run_button, output]))